---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="c0016fd1" -->
# Table des matières
1. [Pourquoi utiliser des métriques de qualité?](#pourquoi-utiliser-des-métriques-de-qualité)
1. [Le coefficient de corrélation R](#le-coefficient-de-corrélation-r)
1. [Préparation des données](#préparation-des-données)
1. [Différentes métriques couramment utilisées](#différentes-métriques-couramment-utilisées)
  1. [Erreur carrée moyenne (*mean squared error* ou MSE)](#erreur-carrée-moyenne-imean-squared-errori-ou-mse)
  1. [Racine de l'erreur carrée moyenne (*root mean squared error* ou RMSE)](#racine-de-lerreur-carrée-moyenne-iroot-mean-squared-errori-ou-rmse)
  1. [R carré (*R-squared* ou $R^2$)](#r-carré-ir-squaredi-ou-r2)
  1. [Erreur absolue médiane (*median absolute error* ou MAE)](#erreur-absolue-médiane-imedian-absolute-errori-ou-mae)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    median_absolute_error,
    r2_score,
)
from sklearn.model_selection import train_test_split

sns.set(color_codes=True)

# Pour la reproductibilité des résultats
seed = 42
np.random.seed(seed)



<!-- #region id="9c5a1d65" -->
Dans ce module, on s'intéresse à quelques métriques utilisées pour évaluer la qualité des régresseurs. Elles sont employées pour comparer divers modèles entre eux afin de choisir le meilleur. Nous aborderons
le sujet de la sélection de modèles plus en détail dans un des modules sur la méthodologie.

Plusieurs métriques sont mentionnées dans cette section, mais seules quelques-unes sont régulièrement utilisées
dans la pratique.
<!-- #endregion -->

<!-- #region id="16d68911" -->
# <a id=pourquoi-utiliser-des-métriques-de-qualité>Pourquoi utiliser des métriques de qualité?</a>
<!-- #endregion -->

<!-- #region id="9852de54" -->
<div align="center">
    <img src= "../images/meeting.jpeg"  width="400" />
    <div>
    <font size="1.5">Image Source: https://www.pngall.com/meeting-png/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="1dd35651" -->
Il arrive souvent au travail que l'on discute d'un problème à résoudre. Par exemple, la façon de traiter un ensemble de données difficilement acquises sur une longue durée. On veut profiter au mieux des efforts et capitaux
investis dans l'aventure. Plusieurs personnes suggèrent alors des solutions, mais
on ne sait pas laquelle choisir. Chacun explique son point de vue en pensant tout haut et en agitant les mains dans les airs
(*hand waving*). Cela arrive régulièrement pendant une séance de réflexion en groupe (*brainstorming*).

En fait, nous sommes entourés de gens compétents et chacun peut regarder le problème
sous un aspect différent. Une telle discussion peut durer longtemps, mais il faut couper court et prendre une décision
pour pouvoir avancer. Pourquoi ne pas faire un test pour décider quelle est la meilleure solution? C'est déjà un
pas en avant, mais encore là les participants peuvent ne pas s'entendre sur la façon de comparer les solutions
entre elles.

La façon la plus pratique est de s'entendre sur une méthode de calcul, une métrique mesurant les performances
de la solution proposée par chacun. On choisit ensuite, parmi toutes les solutions proposées, celle qui optimise la
métrique. Par optimisation, on entend une minimisation ou une maximisation de la métrique.
<!-- #endregion -->

<!-- #region id="b471050a" -->
# <a id=le-coefficient-de-corrélation-r>Le coefficient de corrélation R</a>
<!-- #endregion -->

<!-- #region id="b1ae982a" -->
Avant de discuter des différentes métriques, nous allons présenter le coefficient de corrélation que nous avons déjà utilisé précédemment sans vous l'expliquer. Il s'agit d'un puissant outil de comparaison entre deux ensembles de données $X$ et $Y$. Nous ne l'utilisons pas
comme une métrique, car sa valeur peut être négative et positive. La qualité de la correspondance entre deux ensembles de données est
mieux représentée par la valeur absolue $|R|$, ou encore par le carré de celle-ci $R^2$. Cette dernière métrique est
discutée plus loin dans le document.

> À noter que dans ce qui suit, on utilise la notation standard où $X$ et $Y$ sont deux ensembles de valeurs; ils
ne correspondent pas aux variables $x_{i}$ et à la réponse $y$ qu'on utilise dans la discussion sur la régression.

Le coefficient de corrélation entre deux ensembles de données $X$ et $Y$ se calcule comme suit
$$R = \rho(X,Y) = \dfrac{\frac{1}{n}\sum\limits_{i=1}^{n}(X^{(i)}-\mu_{X})(Y^{(i)}-\mu_{Y})}{\sigma_{X} \sigma_{Y}}$$

où $\mu_{X}$ et $\mu_{Y}$ sont les moyennes des ensembles et $\sigma_{X}$ et $\sigma_{Y}$ sont les écarts-types
des ensembles.

La figure suivante montre plusieurs exemples de distributions de valeurs en $X$ et $Y$ avec les valeurs de $R$ associées:

- la première ligne montre que plus le coefficient $R$ est proche des valeurs extrêmes -1 et 1 (ou que la valeur de $R^2$ est près de 1), mieux définie est la relation linéaire entre les variables,
- la seconde ligne montre que la valeur de $R$ change de signe selon l'orientation du nuage de points,
- la dernière ligne montre qu'une corrélation égale à 0 signifie que les variables ne sont pas corrélées
linéairement, elles peuvent néanmoins être corrélées non linéairement (ex. cercle, sinusoïde, croissant).
<!-- #endregion -->

<!-- #region id="af30ee65" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/cluster-results.png" width="600"  width="200" />
    <div>
    <font size="1.5">Image Source: https://fr.wikipedia.org/wiki/Corrélation_(statistiques)/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="f9879f10" -->
Quelle est l'intuition derrière la formule du coefficient de corrélation? Le numérateur est la partie la plus importante dans la formule. Il est positif lorsque $X$ et $Y$ augmentent en même temps. Il est négatif lorsque $X$ augmente et $Y$ diminue en même temps. Le numérateur tend vers zéro lorsque les données sont très bruitées, c'est-à-dire, $X$ et $Y$ varient de façon chaotique. C'est ce qu'on peut voir dans la figure précédente.

Le coefficient de corrélation R est omniprésent en analyse des données et est particulièrement utile
pour comparer des résultats expérimentaux avec les prédictions de modèles théoriques ou empiriques. Il
est à l'origine de plusieurs métriques de régression dont la plus connue, celle de $R^2$, est décrite plus loin.
<!-- #endregion -->

<!-- #region id="5a62e5e3" -->
# <a id=préparation-des-données>Préparation des données</a>
<!-- #endregion -->

<!-- #region id="fe34fc20" -->
Dans ce qui suit, nous allons faire les opérations suivantes:


- générer un ensemble de données $(x,y)$ où la réponse $y$ est la somme d'une quadratique en
$x$ additionnée de bruit gaussien,
- séparer les données $(x,y)$ en ensembles d'entraînement et de test,
- entraîner plusieurs modèles de régression (des polynômes de degrés 1, 2 et 6) avec l'ensemble d'entraînement,
- prédire les réponses pour les données de test,
- calculer une métrique de performance pour chaque modèle avec les valeurs réelles et prédites en test,
- comparer les valeurs des métriques.
<!-- #endregion -->

<!-- #region id="ff30696e" -->
Nous commençons en générant des données aléatoires pour notre exemple. Le modèle original est un polynôme de degré 2 (quadratique) auquel on ajoute un bruit gaussien (c.-à-d. du bruit aléatoire suivant une loi normale).
<!-- #endregion -->



In [ ]:
# Coefficients du modèle original
c0 = [0.03, 0.2, -2]

# Nombre de données à générer
N = 10

# Écart-type du bruit gaussien
sigma = 0.2

# Plage des données expérimentales
x_min = -2.5
x_max = 10.0
xx = np.linspace(x_min, x_max, 100)[:, np.newaxis]

# Génère les positions aléatoires X des données expérimentales.
X = np.random.uniform(x_min, x_max, N)

# Génère la réponse y des données expérimentales
y = np.polyval(c0, X) + np.random.normal(0.0, sigma, N)



<!-- #region id="68d35ed7" -->
Ici, nous allons séparer les données en deux ensembles de données, l'un pour l'entraînement et le second pour le test. Les données d'entraînement vont servir à entraînement les modèles de régression. C'est-à-dire, à estimer les valeurs des paramètres $a_0, \cdots, a_N$ où $N$ est le degré d'un polynôme de régression. Les données de test vont ensuite servir à mesurer les performances des modèles à prédire la réponse $y$ sans bruit.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)



<!-- #region id="99e6e0fa" -->
Par la suite, nous calculons les coefficients de chaque modèle en effectuant une régression d'un degré donné (voir plus bas).

On effectue une régression sur les données d'entraînement avec la fonction
[`polyfit`](https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html)
 de la librairie Numpy pour les modèles suivants:


- `c1`: polynôme de degré 1,
- `c2`: polynôme de degré 2,
- `c3`: polynôme de degré 6.
<!-- #endregion -->



In [ ]:
c1 = np.polyfit(X_train, y_train, 1)
c2 = np.polyfit(X_train, y_train, 2)
c3 = np.polyfit(X_train, y_train, 6)



<!-- #region id="950c03f5" -->
Calculons les réponses prédites par chaque modèle pour les valeurs X de test.
<!-- #endregion -->



In [ ]:
y_pred1 = np.polyval(c1, X_test)
y_pred2 = np.polyval(c2, X_test)
y_pred3 = np.polyval(c3, X_test)



<!-- #region id="fcf4bc58" -->
Calculons maintenant la réponse prédite par chaque modèle sur la plage des valeurs expérimentales
<!-- #endregion -->



In [ ]:
y0 = np.polyval(c0, xx)
y1 = np.polyval(c1, xx)
y2 = np.polyval(c2, xx)
y3 = np.polyval(c3, xx)



<!-- #region id="07de1fb1" -->
Affichons ensuite les résultats de régression pour chaque modèle.
<!-- #endregion -->



In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(
    2, 2, sharex=True, sharey=True, figsize=(12, 10)
)

# Modèle original
ax1.plot(xx, y0, color="yellow", linewidth=2)
ax1.plot(X_train, y_train, "o", color="black", markersize=5)
ax1.plot(X_test, y_test, "o", color="red", markersize=5)
ax1.set_ylabel("y", rotation=0, fontsize=18)
ax1.set_xlabel("x", fontsize=18)
ax1.yaxis.set_label_coords(-0.1, 0.5)
ax1.xaxis.set_label_coords(0.5, -0.05)
ax1.set_xlim(-3.0, 10.0)
ax1.set_ylim(-3.0, 3.0)
ax1.legend(["Modèle original", "Données Train", "Données Test"], loc="upper left")
ax1.set_facecolor("#BBBBFF")

# Modèle linéaire
ax2.plot(xx, y1, color="black", linewidth=1)
ax2.plot(X_train, y_train, "o", color="black", markersize=5)
ax2.plot(X_test, y_test, "o", color="red", markersize=5)
ax2.set_ylabel("y", rotation=0, fontsize=18)
ax2.set_xlabel("x", fontsize=18)
ax2.yaxis.set_label_coords(-0.1, 0.5)
ax2.xaxis.set_label_coords(0.5, -0.05)
ax2.legend(
    [r"Polynôme de degré 1", "Données Train", "Données Test"],
    loc="upper left",
)
props = dict(boxstyle="round", facecolor="wheat", alpha=0.99)
ax2.set_facecolor("#BBBBFF")

# Modèle quadratique
ax3.plot(xx, y2, color="black", linewidth=1)
ax3.plot(X_train, y_train, "o", color="black", markersize=5)
ax3.plot(X_test, y_test, "o", color="red", markersize=5)
ax3.set_ylabel("y", rotation=0, fontsize=18)
ax3.set_xlabel("x", fontsize=18)
ax3.yaxis.set_label_coords(-0.1, 0.5)
ax3.xaxis.set_label_coords(0.5, -0.1)
ax3.legend(
    [r"Polynôme de degré 2", "Données Train", "Données Test"],
    loc="upper left",
)
ax3.set_facecolor("#BBBBFF")

# Polynôme de degré 6
ax4.plot(xx, y3, color="black", linewidth=1)
ax4.plot(X_train, y_train, "o", color="black", markersize=5)
ax4.plot(X_test, y_test, "o", color="red", markersize=5)
ax4.set_ylabel("y", rotation=0, fontsize=18)
ax4.set_xlabel("x", fontsize=18)
ax4.yaxis.set_label_coords(-0.1, 0.5)
ax4.xaxis.set_label_coords(0.5, -0.1)
ax4.legend(
    [r"Polynôme de degré 6", "Données Train", "Données Test"],
    loc="upper left",
)
ax4.set_facecolor("#BBBBFF")

fig.tight_layout()

y2 = np.polyval(c2, xx)
y3 = np.polyval(c3, xx)



<!-- #region id="36e3d17e" -->
Dans chaque panneau, les points identifient les données expérimentales. Dans le premier panneau, la courbe jaune
correspond au modèle original à partir duquel toutes les données d'entraînement et de test ont été générées. Elles sont
bien réparties autour de la courbe.

Dans les autres panneaux, la courbe noire montre le modèle obtenu par régression en utilisant les données
d'entraînement (les points noirs).

On remarque que la qualité des régressions varie grandement dans les trois derniers panneaux.
Le recalage des données est médiocre dans le second panneau, excellent dans le troisième et médiocre à nouveau dans le quatrième.

On verra dans un des modules sur la méthodologie que ce comportement est relié aux phénomènes très importants du
sous-apprentissage et du surapprentissage.


L'important à retenir est que le modèle `c2` produit les meilleurs résultats en régression. Rien de bien
surprenant puisque les données originales ont été générées avec un modèle quadratique et que ce modèle est lui-même quadratique.

Dans ce qui suit, nous allons montrer comment certaines métriques permettent de sélectionner le modèle optimal. Elles vont toutes identifier le modèle `c2` comme le meilleur.
<!-- #endregion -->

<!-- #region id="59986fde" -->
# <a id=différentes-métriques-couramment-utilisées>Différentes métriques couramment utilisées</a>
<!-- #endregion -->

<!-- #region id="ee4828fa" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/podium-1-2-3.jpeg"  width="300" />
    <div>
    <font size="1.5">Image Source: https://s3.amazonaws.com/freestock-prod/450/freestock_58144363.jpg/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="ce8d65ea" -->
Supposons que l'on ait modélisé une réponse $y$ comme une fonction $f$ dépendant de $x$ et des paramètres $\Theta$.

$$y = f(x,\Theta)$$

La valeur prédite par le modèle pour une donnée $i$ est
$$\hat{y}^{(i)} = f(x^{(i)},\Theta)$$

La qualité d'un recalage doit être mesurée avec une valeur quantitative et non qualitative (« à l'oeil »). C'est pour cela qu'on utilise une méthode de mesure de référence, c'est-à-dire, une métrique.

Certes plusieurs métriques ont été proposées dans la littérature scientifique, mais toutes ne sont pas utilisées régulièrement dans la pratique. Nous allons vous indiquer lesquelles sont les plus courantes dans la pratique. 

Par convention, nous allons également utiliser les acronymes des noms anglais puisqu'ils sont omniprésents dans la littérature.

Il faut toutefois mentionner que plusieurs métriques maison (*in house*) sont conçues et implémentées en milieu industriel. Ces métriques spécialisées permettent d'optimiser des processus basés sur des chaînes d'opérations très complexes.
<!-- #endregion -->

<!-- #region id="eb727cf3" -->
## <a id=erreur-carrée-moyenne-imean-squared-errori-ou-mse>Erreur carrée moyenne (*mean squared error* ou MSE)</a>
<!-- #endregion -->

<!-- #region id="5b42912e" -->
C'est la métrique de base et la plus courante dans la pratique. Elle est très populaire, car elle permet d'obtenir des expressions exactes
pour les paramètres $\Theta$ de certains modèles couramment utilisés. Cette métrique doit être minimisée.

$$\text{MSE} = \frac{1}{n}\sum\limits_{i=1}^{n}(y^{(i)}-\hat{y}^{(i)})^{2}$$
<!-- #endregion -->



In [ ]:
print("Modèle 1: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred1))
print("Modèle 2: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred2))
print("Modèle 3: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred3))



<!-- #region id="0f6560ea" -->
Nous voyons que la valeur de MSE la plus petite est effectivement celle du deuxième modèle. Gardons toutefois en tête que notre jeu de données est très petit (10 données). L'objectif est simplement de démontrer comment effectuer une analyse.
<!-- #endregion -->

<!-- #region id="d2f7aca9" -->
## <a id=racine-de-lerreur-carrée-moyenne-iroot-mean-squared-errori-ou-rmse>Racine de l'erreur carrée moyenne (*root mean squared error* ou RMSE)</a>
<!-- #endregion -->

<!-- #region id="f1ba5778" -->
C'est la racine carrée de la précédente. Elle est plus facilement interprétable. Elle correspond à
l'écart-type de l'erreur de recalage. Cette métrique doit être minimisée.

$$\text{RMSE} = \sqrt{\frac{1}{n}\sum\limits_{i=1}^{n}(y^{(i)}-\hat{y}^{(i)})^{2} }$$
<!-- #endregion -->



In [ ]:
print(
    "Modèle 1: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred1, squared=False)
)
print(
    "Modèle 2: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred2, squared=False)
)
print(
    "Modèle 3: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred3, squared=False)
)



<!-- #region id="50a1c335" -->
## <a id=r-carré-ir-squaredi-ou-r2>R carré (*R-squared* ou $R^2$)</a>
<!-- #endregion -->

<!-- #region id="95c5489c" -->
Probablement la métrique la plus importante et la plus utilisée, car c'est la plus intuitive de toutes! La qualité d'une
régression est souvent déterminée visuellement en affichant les valeurs mesurées de la
réponse $y^{(i)}$ en fonction des valeurs prédites $\hat{y}^{(i)}$. On peut ensuite calculer
le coefficient de corrélation R. Plus R est près de $1$, meilleure est l'adéquation entre les
deux ensemble de données, et donc meilleur est le modèle utilisé.

Une formulation utile de la métrique de $R^2$ est la suivante:

$$R^2 = 1-\frac{\text{SSE}}{\text{SST}}$$


avec la variance des erreurs du modèle correspondant à
$$\text{SSE} = \frac{1}{n-1}\sum\limits_{i=1}^{n}(y^{(i)}-\hat{y}^{(i)})^{2},$$

et la variance totale du signal dans les données correspondant à
$$\text{SST} = \frac{1}{n-1}\sum\limits_{i=1}^{n}(y^{(i)}-\bar{y})^{2}.$$

La valeur de $R^2$ correspond ainsi au pourcentage de la variance totale qui est expliquée par le modèle
de régression. Remplacez le mot variance par information; le modèle explique la fraction
$R^2$ de toute l'information contenue dans les données. C'est beaucoup plus parlant ainsi!
Cette métrique doit être maximisée.
<!-- #endregion -->



In [ ]:
print("Modèle 1: Test R2     : %.3f" % r2_score(y_test, y_pred1))
print("Modèle 2: Test R2     : %.3f" % r2_score(y_test, y_pred2))
print("Modèle 3: Test R2     : %.3f" % r2_score(y_test, y_pred3))



<!-- #region id="c81905c6" -->
Il est possible que la valeur de $R^2$ soit négative. Puisque pour les données d'entraînement, on a toujours $\text{SST}>\text{SSE}$ et donc $R^2>0$. Toutefois, lorsqu'on compare les réponses réelles et prédites en test, il arrive que $\text{SST}<\text{SSE}$ et donc $R^2<0$.

Cela se produit quand le modèle recalé fait des prédictions pires que simplement prédire la moyenne des valeurs de l'ensemble `y_test`. Pour résumer, la valeur de $R^2$ est d'autant plus négative que le modèle fait des prédictions très différentes du signal original.
<!-- #endregion -->

<!-- #region id="b307b66e" -->
## <a id=erreur-absolue-médiane-imedian-absolute-errori-ou-mae>Erreur absolue médiane (*median absolute error* ou MAE)</a>
<!-- #endregion -->

<!-- #region id="899d3842" -->
Les métriques précédentes sont très sensibles aux valeurs aberrantes (*outliers*) dans les données. Ce sujet sera abordé dans un prochain module, mais en bref il s'agit de valeurs très différentes des autres. La qualité d'un recalage est fortement affectée par leur présence.

Les métriques basées sur la médiane sont plus robustes aux valeurs aberrantes. Elles sont toutefois
moins utilisées dans la pratique principalement pour deux raisons. La première est que les gens sont
plus familiers avec la moyenne d'un ensemble de données que sa médiane. La seconde raison est qu'en
début de projet, les données sont nettoyées et la majorité des valeurs aberrantes
sont éliminées à cette étape. Ce sujet est abordé dans un des modules sur le prétraitement des données.
Cette métrique doit être minimisée.

$$\text{MAE} = \text{median}(|y_{1}-\hat{y}_{1}|,\cdots,|y_{n}-\hat{y}_{n}|)$$
<!-- #endregion -->



In [ ]:
print("Modèle 1: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred1))
print("Modèle 2: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred2))
print("Modèle 3: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred3))



<!-- #region id="db83d8cf" -->
Comme on l'a vu, les métriques permettent de départager les modèles de régression utilisés. Elles ont toutes identifié le second modèle comme le meilleur. Toutefois, il arrive, avec des données très bruitées ou aberrantes, que le modèle optimal varie entre les métriques.
<!-- #endregion -->

<!-- #region id="9d1d8a08" -->
Toutefois, comme nous l'avons mentionné, notre jeu de données est composé uniquement de 10 données, ce qui est très petit. L'objectif était surtout de démontrer le fonctionnement des métriques. Nous constatons cependant que si nous augmentons le nombre de données (en testant la valeur de N), les résultats ne pointent pas toujours vers le second modèle. Dépassé un certain seuil (autour de 1 000 données), il n'y a pas de différence entre le second et le troisième modèle. Nous aborderons un peu plus loin dans cette formation comment mieux sélectionner un modèle.
<!-- #endregion -->



In [ ]:
# Nombre de données à générer
N = 1000


In [ ]:
## Procédure d'entraînement et d'évaluation complète

# Génère les nouvelles données
X = np.random.uniform(x_min, x_max, N)
y = np.polyval(c0, X) + np.random.normal(0.0, sigma, N)

# Séparation des données
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

# Entraînement
c1 = np.polyfit(X_train, y_train, 1)
c2 = np.polyfit(X_train, y_train, 2)
c3 = np.polyfit(X_train, y_train, 6)

# Évaluation
y_pred1 = np.polyval(c1, X_test)
y_pred2 = np.polyval(c2, X_test)
y_pred3 = np.polyval(c3, X_test)

y1 = np.polyval(c1, xx)
y2 = np.polyval(c2, xx)
y3 = np.polyval(c3, xx)

print("Modèle 1: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred1))
print("Modèle 2: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred2))
print("Modèle 3: Test MSE     : %.3f" % mean_squared_error(y_test, y_pred3))

print(
    "Modèle 1: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred1, squared=False)
)
print(
    "Modèle 2: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred2, squared=False)
)
print(
    "Modèle 3: Test RMSE     : %.3f"
    % mean_squared_error(y_test, y_pred3, squared=False)
)

print("Modèle 1: Test R2     : %.3f" % r2_score(y_test, y_pred1))
print("Modèle 2: Test R2     : %.3f" % r2_score(y_test, y_pred2))
print("Modèle 3: Test R2     : %.3f" % r2_score(y_test, y_pred3))

print("Modèle 1: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred1))
print("Modèle 2: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred2))
print("Modèle 3: Test MAE     : %.3f" % median_absolute_error(y_test, y_pred3))
